Step 2 converts each of the different flat files into a parquet file. 

This program is split up into 2 components:

The first half defined that work that remains to be done and puts that work into a random order

The second component loops through the work that remains to be done, sees if it hasn't been done yet, and then does it. 

Work is performe in a random order to allow for multiple processes to be executed at once, if work is being performed in the same order then all of the processes were performing the same work at the same time resulting in a great deal of overlapping work.

In [1]:
import pandas as pd
import os
import sys
import random
sys.path.insert(1, './.local/lib/python3.9/site-packages')
sys.path.insert(1, '/users/PAS2164/lawr47/.local/bin')

alarmCol=["patientIdPrimary-id","patientIdPrimary-type","patientIdPrimary-authority","patientNameGiven","patientNameFamily","assignedLocationCareArea","assignedLocationRoom","assignedLocationBed","polltime","alarmName","abnormalFlags","inactivationState","sil",'setLow','setHigh',"chanValue"]
waveCol=["patientIdPrimary-id","patientIdPrimary-type","patientIdPrimary-authority","patientNameGiven","patientNameFamily","assignedLocationCareArea","assignedLocationRoom","assignedLocationBed","polltime","mgname","mgGain","mgHZ","mgwave","mguom","mgsite","mgscale","mginvalid","mgmissing","mgPoints","mgPointsBytes","mgMin","mgMax","mgOffset"]
measureCol=["patientIdPrimary-id","patientIdPrimary-type","patientIdPrimary-authority","patientNameGiven","patientNameFamily","assignedLocationCareArea","assignedLocationRoom","assignedLocationBed","polltime","mesname","msite","muom","mtext"]
inLoc="/fs/ess/scratch/PAS2164/James2019Out/"
outLoc="/fs/ess/scratch/PAS2164/James2019Parquet/"

#Build a list of all parquet files already completed
iList=[]
for pfile in os.listdir(outLoc):
    iList.append(pfile)

#For each file in the infolder
n=0
initList=[]
for outFile in os.listdir(inLoc):
    if outFile.split(".")[-1]=="tsv":
        newFolder=1
        if (outFile.split(".tsv")[0]+'.parquet') in iList:
            newFolder=0
        if newFolder==1:
            initList.append(outFile)

#Ranzomize the list of work.
random.shuffle(initList)

FileNotFoundError: [Errno 2] No such file or directory: '/fs/ess/scratch/PAS2164/James2019Parquet/'

In [ ]:
for outFile in initList:
    #If that file is a csv
    if outFile.split(".")[-1]=="tsv":
        newFolder=1
        #if it's a file that's already been processed, skip it.
        if (outFile.split(".tsv")[0]+'.parquet') in iList:
            newFolder=0
        if newFolder==1:
            if newFolder==1:
                #If this is a new file we're working on...
                #determine the file type to assign the schema.
                fileType = outFile.split("^")[0]
                if fileType=="Wave":
                    columns=waveCol
                if fileType=="Alarm":
                    columns=alarmCol
                if fileType=="Measurement":
                    columns=measureCol
                #Read in the tsv file
                df=pd.read_csv(inLoc+outFile,sep='\t',names=columns,low_memory=False)
                #Remove duplicate values
                dfu=df.drop_duplicates()
                #Output the file as a parquet
                dfu.to_parquet(outLoc+outFile.split(".tsv")[0]+'.parquet', index=False)
                n=n+1
                if n%1000==0:
                    print(str(n),print(outLoc+outFile.split(".tsv")[0]))

/fs/ess/scratch/PAS2164/Ross2019Parquet/Measurement^none^6RH-E~6004^2019-04-22
1000 None


At the end of this program for every flat file that was generated in step 1, there is a parquet file of that flat file in step 2. 